In [1]:
import os
import json
import enum
import logging
from tqdm import tqdm

import sys
sys.path.append('/home/chendian/BBCM/')

from tools.inference import *
from bbcm.config import cfg
from bbcm.utils import get_abs_path
from bbcm.utils import load_json, dump_json

from transformers import BertTokenizer
from bbcm.data.loaders.collator import *

In [2]:
"""
outputs = 
[('张三', 'PERSON'), ('和', 'O'), ('李四', 'PERSON'), ('在', 'O'), ('2019年', 'DATE'), ('3月', 'DATE'), ('23日', 'DATE'), ('在', 'O'), 
('北京', 'STATE_OR_PROVINCE'), ('的', 'O'), ('腾讯', 'ORGANIZATION'), ('技术', 'ORGANIZATION'), ('有限', 'ORGANIZATION'), 
('公司', 'ORGANIZATION'), ('一起', 'O'), ('开会', 'O'), ('。', 'O')]
"""
   
def add_ner_positions_for_samples(collator, csc_sample_file_path, invalid_tags=['PERSON']):
    # set invalid tags
    collator.invalid_tags = invalid_tags

    # 加载 csc samples 文件
    samples = load_json(csc_sample_file_path)

    for i, sample in tqdm(enumerate(samples)):
        text = sample['correct_text']
        word_offsets, ner_mask, chn_mask = collator.get_valid_position_mask(text)

        samples[i]['word_offsets'] = word_offsets
        samples[i]['ner_mask'] = ner_mask
        samples[i]['chn_mask'] = chn_mask
        
    output_file_path = csc_sample_file_path.replace('.json', '.wpos.json')
    dump_json(samples, output_file_path)
    return samples
    

In [3]:
# 加载 Collater
config_file='csc/train_SoftMaskedBert.yml'

cfg.merge_from_file(get_abs_path('configs', config_file))
tokenizer = BertTokenizer.from_pretrained(cfg.MODEL.BERT_CKPT)

from bbcm.data.loaders.dynamic_collator import *
ddc = DynamicDataCollatorForCsc(tokenizer=tokenizer, augmentation=True)

# 10000 sentences / 188 seconds
file_names = [
    # './datasets/csc_py/test.json', 
    # './datasets/csc_py/dev.json', 
    # './datasets/csc_py/train.json',
    "../datasets/csc_aug/sighan15_train.json",
]
for file_path in file_names:
    add_ner_positions_for_samples(collator=ddc, csc_sample_file_path=file_path)


Use beam method.
Pinyin sampling mode: special.
Token sampling mode: sort.
Now loading pinyin2token corpus.
Now loading zi_sim_matrix.
Now loading REDscore.
Now generating score matrix.
Load word freuency data.
Dynamic Data Collator Init
with 5.0% faulty positions.
Now loading NER model from /home/chendian/download/stanford-corenlp-4.2.2/


2339it [01:04, 36.51it/s]
